# Pip Installs


In [ ]:
!pip install ipython-autotime

In [ ]:
!pip install neptune

In [ ]:
!pip install -U neptune-tensorflow-keras

In [ ]:
!pip install pyyaml h5py

In [ ]:
pip install tensorflow-privacy

In [ ]:
!pip install ann_visualizer

In [ ]:
!pip install graphviz

# Imports

In [ ]:
import os
import pickle
import scipy.signal
from scipy import fft
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import tensorflow as tf
tf.__version__
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
# Neu
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow_privacy.privacy.optimizers import dp_optimizer_keras

import tensorflow_privacy

import neptune
from ann_visualizer.visualize import ann_viz;



In [ ]:
from tensorflow.python.client import device_lib

tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
%load_ext autotime

# Data Preprocessing

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
!ls "/content/drive/MyDrive/Masterarbeit/Transformer/data/WESAD"

In [ ]:
DATA_PATH = os.path.realpath("/content/drive/MyDrive/Masterarbeit/Transformer/data/WESAD")
#DATA_PATH = os.chdir("/content/drive/MyDrive/Masterarbeit/Transformer/data/WESAD")
checkpoint_prepath = os.path.realpath("/content/drive/MyDrive/Masterarbeit/Transformer/code/models/")


In [ ]:
class Subject:
    """Subject of the WESAD dataset.
    Subject Class inspired by: https://github.com/WJMatthew/WESAD"""

    def __init__(self, main_path, subject_number):
        self.name = f'S{subject_number}'
        self.subject_keys = ['signal', 'label', 'subject']
        self.signal_keys = ['chest', 'wrist']
        self.chest_keys = ['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']
        self.wrist_keys = ['ACC', 'BVP', 'EDA', 'TEMP']
        with open(os.path.join(main_path, self.name) + '/' + self.name + '.pkl', 'rb') as file:
            self.data = pickle.load(file, encoding='latin1')
        self.labels = self.data['label']

    def get_wrist_data(self):
        """Returns data measured by the E4 Empatica"""

        data = self.data['signal']['wrist']
        return data
    
    def get_subject_dataframe(self):
        """Returns a dataframe with the preprocessed data of the subject"""
        wrist_data = self.get_wrist_data()
        bvp_signal = wrist_data['BVP'][:,0]
        eda_signal = wrist_data['EDA'][:,0]
        acc_x_signal = wrist_data['ACC'][:,0]
        acc_y_signal = wrist_data['ACC'][:,1]
        acc_z_signal = wrist_data['ACC'][:,2]
        temp_signal = wrist_data['TEMP'][:,0]
        # Upsampling data to match BVP data sampling rate using fourier method as described in Paper/dataset
        eda_upsampled = scipy.signal.resample(eda_signal, len(bvp_signal))
        temp_upsampled = scipy.signal.resample(temp_signal, len(bvp_signal))
        acc_x_upsampled = scipy.signal.resample(acc_x_signal, len(bvp_signal))
        acc_y_upsampled = scipy.signal.resample(acc_y_signal, len(bvp_signal))
        acc_z_upsampled = scipy.signal.resample(acc_z_signal, len(bvp_signal))
        label_df = pd.DataFrame(self.labels, columns=['label'])
        label_df.index = [(1 / 700) * i for i in range(len(label_df))] # 700 is the sampling rate of the label
        label_df.index = pd.to_datetime(label_df.index, unit='s')
        data_arrays = zip(bvp_signal, eda_upsampled, acc_x_upsampled, acc_y_upsampled, acc_z_upsampled, temp_upsampled)
        df = pd.DataFrame(data=data_arrays, columns=['BVP', 'EDA', 'ACC_x', 'ACC_y', 'ACC_z', 'TEMP'])
        df.index = [(1 / 64) * i for i in range(len(df))] # 64 = sampling rate of BVP
        df.index = pd.to_datetime(df.index, unit='s')
        df = df.join(label_df)
        df['label'] = df['label'].fillna(method='ffill')
        df.reset_index(drop=True, inplace=True)
        df.drop(df[df['label'].isin([0.0, 4.0, 5.0, 6.0, 7.0])].index, inplace=True)
        df['label'] = df['label'].replace([1.0, 2.0, 3.0], [0, 1, 0])
        df.reset_index(drop=True, inplace=True)
        df = (df-df.min())/(df.max()-df.min()) # Normalize data (no train test leakage since data frame per subject) 
        return df


In [ ]:
s2 = Subject(DATA_PATH, 2)

In [ ]:
s2.get_subject_dataframe()

In [ ]:
def create_subjects_data() -> dict:
    # Create a dictionary with all the subjects and belonging dataframes
    subjects = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
    subjects_data = {}
    for subject_num in subjects:
        subject = Subject(DATA_PATH, subject_num)
        subjects_data[subject.name] = subject.get_subject_dataframe()

    return subjects_data

### Window

1. Creating the windows
2. Create subwindows from the windows
3. Calculate the fft of the subwindows
4. Average the subwindows

![fft](../images/fft.png)

In [ ]:
# Subwindow length of the biosignals
signal_subwindow_dict = {
    'ACC_x': 7,
    'ACC_y': 7,
    'ACC_z': 7,
    'BVP': 30,
    'EDA': 30,
    'TEMP': 35
    }

In [ ]:
# most frequent element in list
def most_common(lst):
    return max(set(lst), key=lst.count)

In [ ]:
def create_windows(df: pd.DataFrame) -> tuple([pd.DataFrame,list]):
    """Creates windows from the dataframe and returns the windows and the labels.
    If the window is assigned to multiple labels, the most common label is chosen for that period.

    Args:
        df (pd.DataFrame): Subject DataFrame

    Returns:
        tuple[pd.DataFrame,list]: Windows representing the activity of the subject in one minute and the corresponding labels.
    """

    window_len = 64 * 60 # fs = 64 and window length in seconds = 60
    windows, labels = zip(*[(df[i:i+window_len], int(most_common(df['label'][i:i+window_len].to_list()))) for i in range(0,df.shape[0],window_len)])
    return windows, labels

In [ ]:
def create_subwindows(df: pd.DataFrame, signal_subwindow_len: int, signal_name: str) -> list:
    """The function creates subwindows from the windows.

    Args:
        df (pd.DataFrame): Windows representing the activity of the subject in one minute.
        signal_subwindow_len (int): Length of the subwindows.
        signal_name (str): Name of the signal.

    Returns:
        list: Subwindows of the signal in the window.
    """
    subwindow_len = 64 * signal_subwindow_len # fs = 64 and sub-window length in seconds = 30
    window_len = 64 * 60 # fs = 64 and window length in seconds = 60
    window_shift = int(64 * 0.25) # fs = 64 and window shift in seconds = 0.25
    subwindows = []

    for i in range(0, window_len, window_shift):
        if i + subwindow_len <= window_len:
            subwindow = df[signal_name][i:i+subwindow_len]
            subwindows.append(subwindow)
    return subwindows
            

In [ ]:
def fft_subwindows(subwindows: list, duration: int, f_s: int) -> list:
    """Calculates the fft of the subwindows.

    Args:
        subwindows (list): C
        duration (int): _description_
        f_s (int): _description_

    Returns:
        list: Fft coefficients of the subwindows.
    """
    freqs= []
    yfs = []
    for subwindow in subwindows:
        y = np.array(subwindow)
        yf = scipy.fft.fft(y)
        l = len(yf)
        N = f_s * duration
        freq = scipy.fft.fftfreq(N, 1/f_s)

        l //= 2
        amps = np.abs(yf[0:l])
        freq = np.abs(freq[0:l])

        # Sort descending amp   
        p = amps.argsort()[::-1]
        freq = freq[p]
        amps = amps[p]

        freqs.append(freq)
        yfs.append(amps)
    return np.asarray(freqs), np.asarray(yfs)
    

In [ ]:
def average_window(subwindows_fft: list) -> list:
    """Calculates the average of the fft coefficients of the subwindows.

    Args:
        subwindows_fft (list): List of fft coefficients of the subwindows.

    Returns:
        list: Average of the fft coefficients of the subwindow for signals.
    """
    len_yfs = len(subwindows_fft[0])
    avg_yfs = []
    for i in range(len_yfs):
        i_yfs = []
        for yf in subwindows_fft:
            try:
                i_yfs.append(yf[i])
            except IndexError:
                pass
        avg_yfs.append(sum(i_yfs)/len(i_yfs))
    return avg_yfs

In [ ]:
def create_preprocessed_subjects_data(subjects_data: dict) -> dict:
# Creates averaged windows for all subjects from dataframes

    subjects_preprosessed_data = {}
    for subject_name, subject_df in subjects_data.items():
        subjects_preprosessed_data[subject_name] = {}
        windows, labels = create_windows(subject_df)
        yfs_per_min_for_signal = {}
        X = []
        for i in range(0,len(windows) - 1):
            for signal in signal_subwindow_dict.keys():

                duration_in_sec = signal_subwindow_dict[signal]

                subwindows = create_subwindows(windows[i], signal_subwindow_len=duration_in_sec, signal_name=signal)
                freqs, yfs = fft_subwindows(subwindows, duration_in_sec, 64)
                yfs_average = average_window(yfs)[:210]
                yfs_per_min_for_signal[signal] = yfs_average
                
            X.append(pd.DataFrame(yfs_per_min_for_signal).T)
        y = list(labels[:len(windows)-1])
        subjects_preprosessed_data[subject_name]['X'] = X
        subjects_preprosessed_data[subject_name]['y'] = y
    
    return subjects_preprosessed_data


In [ ]:
subjects_data = create_subjects_data()
subjects_preprocessed_data = create_preprocessed_subjects_data(subjects_data)

In [ ]:
subjects_preprocessed_data

In [ ]:
def get_subject_window_data(subjects_preprosessed_data: dict) -> tuple([list, list]):
    # Created train and test data for leave one out cross validation
    all_subjects_X = []
    all_subjects_y = []
    for subject_name, subject_data in subjects_preprosessed_data.items():
        all_subjects_X.append(subject_data['X'])
        all_subjects_y.append(subject_data['y'])
    
    return (all_subjects_X, all_subjects_y)


In [ ]:
all_subjects_X, all_subjects_y = get_subject_window_data(subjects_preprocessed_data)

![OS_Sensors](../images/os_sensors.png)

In [ ]:
SMARTWATCH_OS = {
    'E4': ['ACC_x', 'ACC_y', 'ACC_z', 'TEMP', 'EDA', 'BVP'],
    #'Tizen': ['ACC_x', 'ACC_y', 'ACC_z', 'TEMP', 'BVP'],
    #'WearOS_watchOS': ['ACC_x', 'ACC_y', 'ACC_z', 'TEMP'],
    #'Fitbit': ['ACC_x', 'ACC_y', 'ACC_z', 'TEMP', 'EDA'],
    #'PiaOS': ['TEMP', 'EDA', 'BVP']
    }

In [ ]:
def filter_for_smartwatch_os(smartwatch_os_name: str, all_subjects_X: list) -> list:
    # Adjusts the data for the smartwatch os
    all_subjects_X_adjusted_for_smartwatch_os = []
    for subject_data in all_subjects_X:
        subject_adjusted_for_smartwatch_os = []
        for window in subject_data:
            subject_adjusted_for_smartwatch_os.append(window.loc[SMARTWATCH_OS[smartwatch_os_name]])
        all_subjects_X_adjusted_for_smartwatch_os.append(subject_adjusted_for_smartwatch_os)
    return all_subjects_X_adjusted_for_smartwatch_os


# Transformer Model

## Build Model

### Transformer Model


In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
def build_model(
   input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout,
    mlp_dropout
):

    inputs = keras.Input(input_shape)
    x = inputs
    #layers.LSTM(1)
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    #outputs = layers.Dense(num_output_class, activation="sigmoid")(x)
    outputs = layers.Dense(2, activation="sigmoid")(x)

    #outputs = layers.Dense(num_output_class, activation="softmax")(x)
    
    return keras.Model(inputs, outputs)
    #model

In [ ]:
from tensorflow_privacy.privacy.analysis.compute_noise_from_budget_lib import compute_noise as tfp_computer_noise

"""
Calculate noise for given training hyperparameters
"""
def compute_noise(n, batch_size, target_epsilon, epochs, delta, min_noise=1e-5):
  return tfp_computer_noise(n, batch_size, target_epsilon, epochs, delta, min_noise)

"""
Calculate Delta for given training dataset size n
"""
def compute_delta(n):
# delta should be one magnitude lower than inverse of training set size: 1/n
# e.g. 1e-5 for n=60.000
# take 1e-x, were x is the magnitude of training set size
  delta = np.power(10, - float(len(str(n)))) # remove all trailing decimals
  return delta

##Loop

In [ ]:
def setparams(epochs, num_transformer_blocks, mlp_dropout, dropout, num_heads, head_size, dp_selected,l2_norm_clip, noise_multiplier):

  params = {'num_transformer_blocks':num_transformer_blocks, 
          'min_delta_loss':0.001, 
          'epochs':epochs,
          'batch_size':50,
          'lr':0.0001,
          'mlp_dropout': mlp_dropout,
          'dropout': dropout,
          'num_heads':num_heads,
          'ff_dim':4,
          'head_size':head_size,
          'dp_selected':dp_selected,
          'l2_norm_clip':l2_norm_clip,
          'noise_multiplier':noise_multiplier,
          'num_microbatches':1
          }

  
  return params
  


In [ ]:
def trainmodel(run, params):
  # create an empty dictionary to store the number of epochs
  epochs_dict = {}
  max_epochs = 0;
  neptune_callback = NeptuneCallback(run=run, log_model_diagram=True)

  for os, signals in SMARTWATCH_OS.items():
    with tf.device('/device:GPU:0'):
      print(f'\n\n\nSmartwatchOS: {os} - Signals: {signals} - Num. Signals: {len(signals)}')
      print(f'Number of signals: {len(signals)}')
      all_subjects_X_os = filter_for_smartwatch_os(os, all_subjects_X)

    
    groups_set = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
    subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17] # ids for subjects in WESAD dataset
    
    num_signals = len(signals) # Number of signals in the WESAD dataset measured by the empatica e4
    num_output_class = 2 # Number of output classes (2 - non-stress vs stress)
    num_epochs = params["epochs"]

    all_acc_histories = []
    all_loss_histories = []

    

    for i in groups_set:
        test_index = groups_set[i]
        train_index = [x for x in groups_set if x != test_index]
        print(train_index, test_index)
        print(f'SmartwatchOS: {os} - Person: {test_index}')

        X_train = np.concatenate(np.array([all_subjects_X_os[x] for x in train_index], dtype=object))
        y_train = np.concatenate(np.array([all_subjects_y[y] for y in train_index], dtype=object))
        X_test = all_subjects_X_os[test_index]
        y_test = all_subjects_y[test_index]
        
        weight_balance = y_train.tolist().count(0)/y_train.tolist().count(1)

        X_train = np.asarray(X_train)
        #print(X_train)
        X_test = np.asarray(X_test)
        y_train = np.asarray(y_train)
        y_test = np.asarray(y_test)

        y_train = tf.keras.utils.to_categorical(y_train, num_output_class)
        y_test = tf.keras.utils.to_categorical(y_test, num_output_class)

        tf.keras.backend.clear_session()

        #input_shape = (6,210,)
        input_shape = X_train.shape[1:]
        #input_shape = [num_signals, 210, 1]
        print(input_shape)
        model = build_model(
                input_shape,
                head_size=params["head_size"],
                num_heads=params["num_heads"],
                ff_dim=params["ff_dim"],
                num_transformer_blocks=params["num_transformer_blocks"],
                mlp_units=[128],
                mlp_dropout=params["mlp_dropout"],
                dropout=params["dropout"]
                )
        
        if params["dp_selected"] == 0:
          optimizer = keras.optimizers.Adam(learning_rate=params["lr"])  

        elif params["dp_selected"] == 1:
          # Select your differentially private optimizer
          optimizer = tensorflow_privacy.VectorizedDPKerasAdamOptimizer(
                        l2_norm_clip= params["l2_norm_clip"], 
                        noise_multiplier= params["noise_multiplier"], 
                        num_microbatches= params["num_microbatches"], 
                        learning_rate=params["lr"])



        model.compile(
        #loss="sparse_categorical_crossentropy",
        #loss="categorical_crossentropy",
        loss="binary_crossentropy",
        optimizer=optimizer,
        #metrics=["sparse_categorical_accuracy"],
        #metrics=["categorical_accuracy"],
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.BinaryAccuracy()]
        )
        
        
        #build_model(num_signals, num_output_class)

        checkpoint = tf.keras.callbacks.ModelCheckpoint(#filepath=checkpoint_path,  # Path to save the model file
            checkpoint_prepath + f"/{os}/wesad_{os}_binary_s{subject_ids[test_index]}_{num_epochs}.h5",                                            
            monitor="loss", # The metric name to monitor
            save_best_only=True # If True, it only saves the "best" model according to the quantity monitored 
        )


        

        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor="loss",     # Quantity to be monitored.
            min_delta=params["min_delta_loss"],     # Minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than min_delta, will count as no improvement.
            patience=10,        # Number of epochs with no improvement after which training will be stopped.
            restore_best_weights=True
        )

        callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
        #print(X_train.shape)
        
        history = model.fit(
        X_train,
        y_train,
        validation_split=0.2,
        epochs=num_epochs,
        batch_size=params["batch_size"],
        class_weight={0: 1, 1: weight_balance},
        #callbacks=[checkpoint, callbacks, neptune_callback]
        callbacks=[neptune_callback]         # No Early Stopping, No Checkpoint
        #callbacks=[checkpoint, early_stopping]
        #callbacks=[checkpoint, early_stopping, neptune_callback] #  Early Stopping
        #callbacks=[early_stopping, neptune_callback] #  Early Stopping
        #callbacks=[neptune_callback] #  Early Stopping
        )

        #Getting actual number of epochs per Subject (due to early stopping it differs)
        number_of_epochs_it_ran = len(history.history['loss'])
        n_epochs_best = number_of_epochs_it_ran
        # n_epochs_best = number_of_epochs_it_ran - 10 #subtracting patience when using early stopping
        #Rewrite max_epochs if current Epoch is higher
        if n_epochs_best > max_epochs:
          max_epochs = n_epochs_best
        # Add number of epochs to dictionary for each subject != the current used subject
        for other_subject in train_index:
          if other_subject not in epochs_dict:
            epochs_dict[other_subject] = n_epochs_best
          else:
            epochs_dict[other_subject] += n_epochs_best
        print(epochs_dict)
      
      


    eval_metrics =  model.evaluate(X_test, y_test, verbose=0 )
    loss = model.evaluate(X_test, y_test, verbose=0 )[0]
    accuracy = model.evaluate(X_test, y_test, verbose=0 )[1]
    precision = model.evaluate(X_test, y_test, verbose=0 )[2]
    recall = model.evaluate(X_test, y_test, verbose=0 )[3]
    if (precision + recall) != 0:
      f1 = 2 * precision * recall / (precision + recall)
    else:
      f1 = 0

    
    run["eval/loss"] = loss
    run["eval/accuracy"] = accuracy
    run["eval/precision"] = precision
    run["eval/recall"] = recall
    run["eval/f1"] = f1
    run["eval/maxepochs"] = max_epochs
  return model

In [ ]:
def losoeval(model, run, params):
  # Evaluating every models on the corresponding test dataset not seen during training.
  os_scores_acc = {}
  os_scores_f1 = {}

  for os, signals in SMARTWATCH_OS.items():

    all_subjects_X_os = filter_for_smartwatch_os(os, all_subjects_X)

    all_accuracies = []
    all_precisions = []
    all_recalls = []
    all_f1s = []
    subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17] # ids for subjects in WESAD dataset
    
    for i, subject_id in enumerate(subject_ids):
        X_test = all_subjects_X_os[i]
        y_test = all_subjects_y[i]
        X_test = np.asarray(X_test)
        y_test = np.asarray(y_test)
        
        y_test = tf.keras.utils.to_categorical(y_test, 2)
        
                                            
          
        accuracy = model.evaluate(X_test, y_test, verbose=0 )[1]
        precision = model.evaluate(X_test, y_test, verbose=0 )[2]
        recall = model.evaluate(X_test, y_test, verbose=0 )[3]
        if (precision + recall) != 0:
          f1 = 2 * precision * recall / (precision + recall)
        else:
          f1 = 0
        all_accuracies.append(accuracy)
        all_precisions.append(precision)
        all_recalls.append(recall)
        all_f1s.append(f1)

    print(f'Smartwatch OS: {os}')
    print(f'Evaluation of Transformer model trained on {params["epochs"]} epochs\n')
    print(f'Subject\t\t Accuracy\tPrecision\tRecall\t\tF1-Score')
    print("************************************************************************")
    for i in range(len(all_accuracies)):
        print(f'S{subject_ids[i]}\t\t {round(all_accuracies[i], 5):.5f}\t{round(all_precisions[i], 5):.5f}\t\t{round(all_recalls[i], 5):.5f}\t\t{round(all_f1s[i], 5):.5f}')

    print("************************************************************************")
    print(f'Average\t\t {round(np.mean(all_accuracies), 5):.5f}\t{round(np.mean(all_precisions), 5):.5f}\t\t{round(np.mean(all_recalls), 5):.5f}\t\t{round(np.mean(all_f1s), 5):.5f}\n\n\n')

    os_scores_acc[os] = all_accuracies
    os_scores_f1[os] = all_f1s
  run["loso/accuracy"] = round(np.mean(all_accuracies), 5)
  run["loso/precision"] = round(np.mean(all_precisions), 5)
  run["loso/recall"] = round(np.mean(all_recalls), 5)
  run["loso/f1"] = round(np.mean(all_f1s), 5)
  

  run.stop()


## Loopstart

In [ ]:
dic_numencblocks= [1,2,3,4,5,6,7]
dic_mlpdropout= [0.6,0.6,0.6,0.6,0.6]
dic_dropout= [0.4,0.4,0.4,0.4,0.4,0.5,0.5,0.5,0.5,0.5,0.6,0.6,0.6,0.6,0.6,0.7,0.7,0.7,0.7,0.7]
dic_heads = [5,5,5]
dic_headsize = [512]
dic_epochs = [110]
dic_noise = [68.61306896515201,68.61306896515201,68.61306896515201,68.61306896515201,68.61306896515201]

index = 0;
run = neptune.init_run( api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmNDZjODdkNS1hMGQzLTQ1M2QtYTBhNy03Y2VlYTU2ZWIzZTkifQ==", project="boris/stresstransformer")
from neptune.integrations.tensorflow_keras import NeptuneCallback
for noise in dic_noise:
  if index != 0:
    run = neptune.init_run( api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmNDZjODdkNS1hMGQzLTQ1M2QtYTBhNy03Y2VlYTU2ZWIzZTkifQ==", project="boris/stresstransformer")

  index = index +1
  params = setparams(110, 8, 0.25, 0.25, 4, 256, 1, 2.0, noise)
  run['hyper-parameters'] = params
  model = trainmodel(run,params)
  losoeval(model, run, params)
  


  
  